In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/final/'
sys.path.append(project_dir+'pages/')

Mounted at /content/drive


In [3]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/project/final/pages/signup_page.py"
"""
Sign-up page for the music recommender app.
"""
import sys
import importlib

# Import modules with reload to avoid circular dependencies
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/app')
import layout
importlib.reload(layout)

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/app')
import navigation
importlib.reload(navigation)
from navigation import navigate_to, go_back, show_loading, show_status, get_output

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/app')
import session_state
importlib.reload(session_state)
from session_state import session_state

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/handlers')
import signup_handler
importlib.reload(signup_handler)
from signup_handler import SignUpHandler

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/services')
import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/pages')
import base_page
importlib.reload(base_page)
from base_page import BasePage

import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def render_page():
    """Render the sign-up page."""
    # Create services
    user_manager_instance = UserManager("/content/drive/MyDrive/Colab Notebooks/project/final/data/users.csv")

    # Create handlers
    signup_handler_instance = SignUpHandler(user_manager_instance)

    # Create page
    page = SignUpPage(signup_handler_instance)
    page.render()

class SignUpPage(BasePage):
    """Sign-up page implementation."""

    def __init__(self, signup_handler):
        """
        Initialize the sign-up page.

        Args:
            signup_handler: Instance of SignUpHandler
        """
        super().__init__(title="🔐 Sign Up")
        self.signup_handler = signup_handler

        # Form fields
        self.username_field = None
        self.password_field = None
        self.firstname_field = None
        self.lastname_field = None
        self.age_field = None
        self.gender_field = None

        # Artist selection
        self.selected_artists = []
        self.artist_input = None
        self.artist_box = None

        # Load artist list
        try:
            songs_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project/final/data/songs.csv")
            self.artist_list = sorted(songs_df['artist'].dropna().unique().tolist())
        except:
            self.artist_list = []

    def render(self):
        """Render the sign-up page."""
        # Add title
        self.add_title()

        # Create form fields
        form_style = {'description_width': '100px'}
        self.username_field = widgets.Text(description='Username', style=form_style)
        self.password_field = widgets.Password(description='Password', style=form_style)
        self.firstname_field = widgets.Text(description='First Name', style=form_style)
        self.lastname_field = widgets.Text(description='Last Name', style=form_style)
        self.age_field = widgets.IntText(description='Age', style=form_style)
        self.gender_field = widgets.Dropdown(options=['M', 'F'], description='Gender', style=form_style)

        # Create artist selection fields
        self.add_subtitle("🎵 Select Favorite Artists")

        self.artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=self.artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )

        add_button = widgets.Button(
            description='Add',
            layout=widgets.Layout(width='80px')
        )

        # Artist list container
        self.artist_box = widgets.VBox(layout=widgets.Layout(
            width='100%',
            max_width='400px',
            margin='10px auto'
        ))

        # Add artist handler
        def on_add(b):
            val = self.artist_input.value.strip()
            if val in self.artist_list and val not in self.selected_artists:
                self.selected_artists.append(val)
                self.refresh_artist_table()
                self.artist_input.value = ''
            elif val in self.selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in self.artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        # Registration handler
        def on_signup(b):
            show_loading("Creating account...")
            self.signup_handler.register_user(
                self.username_field.value,
                self.password_field.value,
                self.firstname_field.value,
                self.lastname_field.value,
                self.age_field.value,
                self.gender_field.value,
                self.selected_artists
            )

        # Create buttons
        signup_button = layout.create_button(
            description='Create Account',
            button_style='primary',
            on_click=on_signup
        )

        back_button = layout.create_button(
            description='Back',
            on_click=lambda b: go_back()
        )

        # Create form container
        user_form = widgets.VBox([
            self.username_field,
            self.password_field,
            self.firstname_field,
            self.lastname_field,
            self.age_field,
            self.gender_field
        ], layout=layout.form_layout)

        # Artist input container
        artist_input_container = widgets.HBox(
            [self.artist_input, add_button],
            layout=widgets.Layout(width='100%', max_width='400px', margin='0 auto', justify_content='space-between')
        )

        # Buttons container
        buttons = layout.create_buttons_container([
            signup_button,
            back_button
        ])

        # Add widgets to page
        self.add_widget(user_form)
        self.add_spacer(15)
        self.add_widget(artist_input_container)
        self.add_widget(self.artist_box)
        self.add_spacer(10)
        self.add_widget(buttons)
        self.add_widget(get_output())

        # Display the page
        self.display()

    def refresh_artist_table(self):
        """Refresh the list of selected artists."""
        rows = []
        for a in self.selected_artists:
            label = widgets.Label(a, layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
            remove_btn = widgets.Button(
                description='Remove',
                button_style='danger',
                layout=widgets.Layout(width='80px')
            )

            def make_remove_callback(artist):
                def remove_callback(b):
                    self.selected_artists.remove(artist)
                    self.refresh_artist_table()
                return remove_callback

            remove_btn.on_click(make_remove_callback(a))

            rows.append(widgets.HBox([
                label, remove_btn
            ], layout=widgets.Layout(justify_content='space-between', width='100%')))

        self.artist_box.children = rows

Overwriting /content/drive/MyDrive/Colab Notebooks/project/final/pages/signup_page.py
